In [ ]:
import requests
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import ttk
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Define the Generator network
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# Define the Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define hyperparameters
latent_dim = 100  # Size of the latent space
output_dim = 784  # Size of the generated fashion design (e.g., flattened image size)
batch_size = 32
num_epochs = 100

# Initialize the generator and discriminator
generator = Generator(latent_dim, output_dim)
discriminator = Discriminator(output_dim)

# Define loss function and optimizers
criterion = nn.BCELoss()
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
dis_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)

# Function to generate a random fashion design
def generate_fashion_design():
    # Generate random noise vector
    noise = torch.randn(1, latent_dim)

    # Generate a fashion design using the generator
    generated_design = generator(noise)

    # Reshape the generated design to the desired shape (e.g., image shape)
    generated_design = generated_design.view(-1, 28, 28)

    # Convert the generated design to a numpy array
    generated_design = generated_design.detach().numpy()

    # Plot the generated design
    plt.imshow(generated_design[0], cmap='gray')
    plt.axis('off')
    plt.show()

# Scrape fashion data from Vogue website
def scrape_fashion_data():
    url = 'https://www.vogue.com/fashion-shows'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract years
    years = ['2022']
    year_tags = soup.select('.directory-years-container .directory-years-list .directory-years-link')
    for year_tag in year_tags:
        years.append(year_tag.text.strip())

    # Extract brands
    brands = ['chanel']
    brand_tags = soup.select('.directory-designers-container .directory-designers-list .directory-designers-link')
    for brand_tag in brand_tags:
        brands.append(brand_tag.text.strip())

    # Extract seasons
    seasons = ['spring', 'summer', 'fall', 'winter']

    return years, brands, seasons

# Create a Tkinter window
window = tk.Tk()
window.title("FashionAlchemy")
window.geometry("400x300")

# Scrape fashion data from Vogue website
year_options, brand_options, season_options = scrape_fashion_data()

# Create the UI elements
year_label = ttk.Label(window, text="Year:")
year_label.pack()
year_var = tk.StringVar()
year_dropdown = ttk.Combobox(window, textvariable=year_var, values=year_options)
year_dropdown.pack()

brand_label = ttk.Label(window, text="Brand:")
brand_label.pack()
brand_var = tk.StringVar()
brand_dropdown = ttk.Combobox(window, textvariable=brand_var, values=brand_options)
brand_dropdown.pack()

season_label = ttk.Label(window, text="Season:")
season_label.pack()
season_var = tk.StringVar()
season_dropdown = ttk.Combobox(window, textvariable=season_var, values=season_options)
season_dropdown.pack()

generate_button = ttk.Button(window, text="Generate", command=generate_fashion_design)
generate_button.pack()

# Start the Tkinter event loop
window.mainloop()

